![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

# Project Instructions
The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_clean that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:
- Columns containing **integers** must be stored as 32-bit integers (`int32`).
- Columns containing **floats** must be stored as 16-bit floats (`float16`).
- Columns containing **nominal** categorical data must be stored as the `category` data type.
- Columns containing **ordinal categorical** data must be stored as `ordered categories`, with an order that reflects the natural order of the column.
- The columns of `ds_jobs_clean` must be in the same order as the original dataset.
- The DataFrame should be filtered to only contain students with **10 or more years of experience** at companies with **at least 1000 employees**, as their recruiter base is suited to more experienced professionals at enterprise companies.
- The final line must be `ds_jobs_clean.head(100)` to return only the first 100 rows.

# Solution
## Import and Inspect Dataset

In [12]:
# Start your code here!
import pandas as pd
from pandas.api.types import CategoricalDtype

# import dataset and save columns
df = pd.read_csv('customer_train.csv')
columns = df.columns

# inspect dataset
display(df.head())
display(df.info())
display(df.describe())
display(df.describe(exclude='number'))

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

None

,student_id,city_development_index,training_hours,job_change
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


,city,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,19158,14650,19158,18772,18698,16345,19093,13220,13018,18735
unique,123,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,4355,13221,13792,13817,11598,14492,3286,3083,9817,8040


In [4]:
# Check the unique values of 'object' data types with unique values less than 25
object_cols = ['gender', 'relevant_experience', 'enrolled_university', 
               'education_level', 'major_discipline', 'experience', 
               'company_size', 'company_type', 'last_new_job']

# print unique values
for col in object_cols:
    print(col, ': ', [*df[col].unique()])

gender :  ['Male', nan, 'Female', 'Other'] 

relevant_experience :  ['Has relevant experience', 'No relevant experience'] 

enrolled_university :  ['no_enrollment', 'Full time course', nan, 'Part time course'] 

education_level :  ['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School'] 

major_discipline :  ['STEM', 'Business Degree', nan, 'Arts', 'Humanities', 'No Major', 'Other'] 

experience :  ['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4', '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan] 

company_size :  [nan, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10-49', '100-499', '500-999'] 

company_type :  [nan, 'Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other', 'Public Sector', 'NGO'] 

last_new_job :  ['1', '>4', 'never', '4', '3', '2', nan] 



## Filter and Define `ds_jobs_clean`

**Applaying filters**:
- `experience` of **10** years or more: (note that we have a value of `>20` also inside `experience`).
- `company_size` at least have **1000** employee: so values will be: `1000-4999`, `5000-9999` and `10000+`.

In [9]:
# lets start with filters
experience_filter = [str(num) for num in range(10, 21)] + ['>20']
company_size_filter = ['1000-4999', '5000-9999', '10000+']

# define ds_jobs_clean
ds_jobs_clean = df[(df['experience'].isin(experience_filter)) 
                   & (df['company_size'].isin(company_size_filter))]

# inspact unique values after filtering
for col in object_cols:
    print(col, ': ', [*ds_jobs_clean[col].unique()])

gender :  [nan, 'Male', 'Female', 'Other']
relevant_experience :  ['Has relevant experience', 'No relevant experience']
enrolled_university :  ['no_enrollment', 'Part time course', 'Full time course', nan]
education_level :  ['Graduate', 'Masters', 'Phd', 'High School', nan, 'Primary School']
major_discipline :  ['STEM', 'Business Degree', 'No Major', 'Arts', nan, 'Other', 'Humanities']
experience :  ['17', '>20', '19', '12', '16', '14', '13', '15', '10', '18', '20', '11']
company_size :  ['10000+', '1000-4999', '5000-9999']
company_type :  ['Pvt Ltd', 'Public Sector', nan, 'NGO', 'Other']
last_new_job :  ['>4', '3', '2', '4', '1', 'never', nan]


## Change Data types

**Based on** the inspection of `ds_jobs_clean` we can see that:
- `Integer` columns are: `student_id`, `training_hours`, `job_change`.
- `Float` columns are: `city_development_index`.
- Nomianl `categorical` columns are: `gender`, `major_discipline`, `company_type`.
- Ordinal `categorical` columns are: `relevant_experience`, `enrolled_university`, `education_level`, `experience`, `company_size`, `last_new_job`.

In [11]:
# define columns to change types
int_cols = ['student_id', 'training_hours', 'job_change']
float_cols = ['city_development_index']
nom_cat_cols = ['gender', 'major_discipline', 'company_type']

# for ordinal categorical, we will define dictionary with the orders
ord_cat_cols = {'relevant_experience': ['No relevant experience', 
                                        'Has relevant experience'],
                'enrolled_university': ['no_enrollment', 
                                        'Part time course', 
                                        'Full time course'],
                'education_level': ['Primary School', 'High School',
                                    'Graduate', 'Masters', 'Phd'],
                'experience': experience_filter,
                'company_size': company_size_filter,
                'last_new_job': ['never', '1', '2', '3', '4', '>4']
               } 

In [13]:
# convert data types
# integer
for col in int_cols:
    ds_jobs_clean[col] = ds_jobs_clean[col].astype('int32')

# float
ds_jobs_clean[float_cols[0]] = ds_jobs_clean[float_cols[0]
                                            ].astype('float16')  

# nominal categorical
for col in nom_cat_cols:
    ds_jobs_clean[col] = ds_jobs_clean[col].astype('category')
    
# ordinal categorical
for col in ord_cat_cols.keys():
    ds_jobs_clean[col] = ds_jobs_clean[col].astype(
        CategoricalDtype(categories=ord_cat_cols[col], ordered=True))    

## Submission

In [14]:
ds_jobs_clean.head(100)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,28453,city_16,0.910156,Male,Has relevant experience,no_enrollment,Masters,STEM,10,10000+,Pvt Ltd,1,18,0
931,6304,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,1000-4999,Public Sector,>4,142,0
933,31368,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,10,1000-4999,Pvt Ltd,1,314,0
959,21276,city_75,0.938965,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,4,94,0
